In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from pydoc import help  # can type in the python console `help(name of function)` to get the documentation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML
from pandas.tools.plotting import scatter_matrix
from matplotlib.artist import setp

plt.rcParams['figure.figsize'] = (16,16)

def mosthighlycorrelated(mydataframe, numtoreport):
    # find the correlations
    cormatrix = mydataframe.corr()
    # set the correlations on the diagonal or lower triangle to zero,
    # so they will not be reported as the highest ones:
    cormatrix *= np.tri(*cormatrix.values.shape, k=-1).T
    # find the top n correlations
    cormatrix = cormatrix.stack()
    cormatrix = cormatrix.reindex(cormatrix.abs().sort_values(ascending=False).index).reset_index()
    # assign human-friendly names
    cormatrix.columns = ["FirstVariable", "SecondVariable", "Correlation"]
    return cormatrix.head(numtoreport)

In [2]:
df1=pd.read_csv("DataNN.txt")
df2 = df1.copy()
classes = df1.iloc[:,-1].copy()
df1 = df1.iloc[:,0:-1].copy()
df1 = (df1-np.min(df1, axis = 0))/(np.max(df1, axis = 0)-np.min(df1, axis = 0))+0.01 #Boxcox values must be positive

df1

,Mean_FCC1,Mean_FCC2,Mean_FCC3,Mean_FCC4,Mean_FCC5,Mean_FCC6,Mean_FCC7,Mean_FCC8,Mean_FCC9,Mean_FCC10,...,DER2_FCC7,DER2_FCC8,DER2_FCC9,DER2_FCC10,DER2_FCC11,DER2_FCC12,LENGTH,MINFREC,MAXFREC,DOMFREC
0,0.632988,0.515992,0.613939,0.649391,0.631306,0.537491,0.634551,0.471266,0.558806,0.658142,...,0.466064,0.442061,0.484131,0.374384,0.389453,0.645616,0.118361,0.278896,0.274443,0.273514
1,0.529368,0.424542,0.659519,0.669014,0.689505,0.534587,0.672499,0.461547,0.588286,0.652710,...,0.476477,0.440048,0.487394,0.373674,0.476390,0.646326,0.088058,0.270113,0.263225,0.273382
2,0.768549,0.494183,0.683725,0.568778,0.618954,0.493537,0.568190,0.473348,0.560459,0.645112,...,0.418386,0.487001,0.527227,0.379560,0.429362,0.640440,0.224416,0.272626,0.318057,0.328636
3,0.652105,0.498720,0.622784,0.644085,0.622467,0.527164,0.639839,0.457097,0.566322,0.651621,...,0.476537,0.430084,0.487910,0.370900,0.516596,0.649100,0.095635,0.275133,0.275684,0.270925
4,0.739468,0.518449,0.562540,0.576624,0.767742,0.645997,0.754889,0.533365,0.635321,0.661008,...,0.356339,0.391220,0.482569,0.347319,0.439830,0.672681,0.231992,0.257574,0.301859,0.275393
5,0.699361,0.539567,0.615434,0.589822,0.586202,0.575420,0.656744,0.434573,0.559276,0.648536,...,0.456820,0.446174,0.492594,0.354270,0.438744,0.665730,0.148662,0.273876,0.301859,0.282268
6,0.728678,0.486343,0.685584,0.559428,0.565770,0.384487,0.619700,0.463736,0.546713,0.603656,...,0.495056,0.563372,0.484658,0.377216,0.412910,0.642784,0.148662,0.273876,0.314318,0.325169
7,0.424710,0.459238,0.638172,0.650230,0.662090,0.527701,0.658784,0.458760,0.578890,0.651859,...,0.464227,0.443582,0.483293,0.374880,0.281516,0.645120,0.027457,0.272626,0.260734,0.269594
8,0.774430,0.497860,0.714764,0.583450,0.510752,0.454401,0.454517,0.450339,0.650525,0.649824,...,0.411117,0.351517,0.509836,0.347177,0.453904,0.672823,0.171389,0.272626,0.328026,0.281010
9,0.597230,0.361578,0.595910,0.671893,0.699572,0.545081,0.672957,0.472994,0.597286,0.662166,...,0.516977,0.463886,0.488609,0.371101,0.432402,0.648899,0.065334,0.266350,0.271952,0.280637


In [3]:
NumVariables = df1.shape[1]
NumVariables

76

In [19]:
X = np.array(df1)
y = np.array(classes)-1.0
X

array([[0.63298803, 0.51599235, 0.61393933, ..., 0.27889558, 0.27444258,
        0.27351356],
       [0.52936796, 0.42454158, 0.65951892, ..., 0.2701128 , 0.26322483,
        0.27338192],
       [0.76854902, 0.4941828 , 0.6837249 , ..., 0.27262634, 0.31805746,
        0.32863559],
       ...,
       [0.5305324 , 0.43754136, 0.59894457, ..., 0.02158933, 0.01231325,
        0.01636277],
       [0.5275255 , 0.41245649, 0.55881469, ..., 0.01916931, 0.01338202,
        0.01630279],
       [0.47235481, 0.42634786, 0.59966215, ..., 0.02145488, 0.01338202,
        0.0166224 ]])

In [32]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def d_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

def clearList(L): #arroja una lista de L elementos vacios, no sabia de qué otra forma hacerla
    listica = []
    for i in range(0,L):
        listica.append([]);
    return listica

def RedNeuronalTrain(X,y,Lsizes,alpha = .1,epochs = 20,outputs = 2):
    N  = X.shape[0]   #número de muestras para realizar el entrenamiento
    nf = X.shape[1]   #número de características
    L = len(Lsizes)   #número de capas ocultas
    w_ = []           #pesos de la red neuronal
    
    w_.append(np.random.rand(Lsizes[0],nf+1)) #Primera capa: L_1 filas & nf + 1 columnas (añadiendo la neurona de BIAS)
    for k in range(0,L-1):
        w_.append(np.random.rand(Lsizes[k+1],(Lsizes[k]+1)))#capa siguiente en las filas y capa actual columnas
    w_.append(np.random.rand(outputs,Lsizes[L-1]+1))        #Ultima capa interna: ourput_N flas & L_final columnas 
                                                            #(añadiendo la neurona de BIAS) 
    for j in range(0,epochs): #condición de parada
        J = 0                 #error cuadratico medio  
        for i in range(0,N):
            #One hot encoding
            Y = np.zeros(shape=(outputs,1))
            Y[int(y[i])] = 1.0
            a_ = []
            z_ = []
            #Forward
            #Para cada muestra
            a_.append(X[i,None].T)
            #Se agrega el bias de la capa entrada
            for k in range(0,(L+1)):
                #Se agrega el bias a la capa externa k+1
                a_[k] = np.insert(a_[k], 0, 1,axis=0)
                z_.append(np.dot(w_[k],a_[k]))
                a_.append(sigmoid(z_[k]))
            #print(a_)
            #Backward
            J = J + np.sum((Y-a_[len(a_)-1])*(Y-a_[len(a_)-1]))
            d_=clearList(len(a_))
            
            d_[L+1]= (Y-a_[L+1])*d_sigmoid(z_[L])
            for k in range(L,0,-1):
                d_[k] = np.dot(w_[k].T,d_[k+1])
                d_[k] = np.delete(d_[k],0,axis=0)
                d_[k] = d_[k]*d_sigmoid(z_[k-1])
                
            DE_dw_=clearList(len(w_))
            for k in range(0,len(w_)):
                DE_dw_[k] = -np.dot(d_[k+1],a_[k].T)
            for k in range(0,len(w_)):
                w_[k] = w_[k]-alpha*DE_dw_[k]
            #print(np.array(Y) == np.array(a_[len(a_)-1]))
        #print(a_[len(a_)-1])
        print(J/2)
    return [w_]

def Predict(X,w_): 
    #Forward
    #Para cada muestra
    L = len(w_)
    a_ = []
    z_ = []
    a_.append(X[:,None])
    for k in range(0,(L)):
        #Se agrega el bias a la capa externa k+1
        a_[k] = np.insert(a_[k], 0, 1,axis=0)
        z_.append(np.dot(w_[k],a_[k]))
        a_.append(sigmoid(z_[k]))
    return a_[len(a_)-1]


In [33]:
index=np.random.permutation(X.shape[0])
X1    = X[index,:]
y1    = y[index  ]

Nsamples = X.shape[0]
Ntrain   = int(0.7*Nsamples)
Nval     = int(0.15*Nsamples)
Ntest    = Nsamples-Ntrain-Nval

X_train = X1[0:Ntrain,:]
X_val   = X1[Ntrain:(Ntrain+Nval),:]
X_test  = X1[(Nsamples-Ntest):Nsamples,:]
y_train = y1[0:Ntrain  ]
y_val   = y1[Ntrain:(Ntrain+Nval)  ]
y_test  = y1[(Nsamples-Ntest):Nsamples  ]

### Entrenamiento de la red neuronal usando validación bootstraping

In [34]:
sizes = np.array([31,31,31,31,15,15,11])
[w_] = RedNeuronalTrain(X_train,y_train,Lsizes=sizes,outputs=12,alpha=0.1)

3776.1629691104004
1067.5513402747117
532.0685330566555
531.9806324091562
531.9725104546869
531.9707233871053
531.9701789451395
531.9699645093237
531.9698567867373
531.9697890211871
531.9697377369208
531.9696935833
531.9696525145737
531.969612692643
531.9695732590075
531.9695338056529
531.9694941356082
531.9694541516628
531.969413803925
531.969373064966


In [35]:
predicted1 = []
for i in range(0,X_train.shape[0]):
    predicted1.append(np.argmax(Predict(X_train[i],w_)))
Accuracy_train = np.sum(np.array(predicted1) == y_train)/X_train.shape[0]
print('error de entrenamiento = '+str(Accuracy_train))

error de entrenamiento = 0.5139171758316361


In [36]:
predicted2 = []
for i in range(0,X_val.shape[0]):
    predicted2.append(np.argmax(Predict(X_val[i],w_)))
Accuracy_val = np.sum(np.array(predicted2) == y_val)/X_val.shape[0]
print('error de validación = '+str(Accuracy_val))

error de validación = 0.4634920634920635


In [37]:
d = {'2. predicción': np.array(predicted1),'1. label': y_train, '3. equals':np.array(predicted1) == y_train}
df = pd.DataFrame(data=d)
df

,1. label,2. predicción,3. equals
0,0.0,0,True
1,0.0,0,True
2,0.0,0,True
3,0.0,0,True
4,0.0,0,True
5,0.0,0,True
6,10.0,0,False
7,0.0,0,True
8,4.0,0,False
9,4.0,0,False
